Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats
from secrets import IEX_CLOUD_API_TOKEN

Import List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

Split List into groups for batch API calls

In [3]:
def split(list, n):
    for i in range(0, len(list), n):
        yield list[i : i+n]

groups = list(split(stocks['Ticker'], 100))
stock_symbols = []

for i in range(0, len(groups)):
    stock_symbols.append(','.join(groups[i]))

Create dataframe to store results

In [4]:
data_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'Momentum Score'
                ]

momentum_dataframe = pd.DataFrame(columns = data_columns)

Store API data in dataframe, using placeholder "none" for uncalcalculated data

In [5]:
for batch in stock_symbols:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    stock_data = requests.get(batch_api_call_url).json()
    
    for stock in batch.split(','):
        momentum_dataframe = momentum_dataframe.append(
                                        pd.Series([stock, 
                                                   stock_data[stock]['quote']['latestPrice'],
                                                   'none',
                                                   stock_data[stock]['stats']['year1ChangePercent'],
                                                   'none',
                                                   stock_data[stock]['stats']['month6ChangePercent'],
                                                   'none',
                                                   stock_data[stock]['stats']['month3ChangePercent'],
                                                   'none',
                                                   stock_data[stock]['stats']['month1ChangePercent'],
                                                   'none',
                                                   'none'
                                                   ], 
                                                  index = data_columns), 
                                        ignore_index = True)

momentum_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,152.250,none,0.725821,none,0.257105,none,0.219287,none,0.103362,none,none
1,AAL,22.343,none,0.737275,none,0.437642,none,0.024899,none,-0.029083,none,none
2,AAP,207.122,none,0.430485,none,0.260278,none,0.116394,none,0.059896,none,none
3,AAPL,137.260,none,0.501891,none,0.014254,none,0.114543,none,0.052123,none,none
4,ABBV,113.230,none,0.275424,none,0.144046,none,0.132897,none,-0.010288,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.800,none,0.386107,none,0.100533,none,0.098295,none,-0.025591,none,none
501,ZBH,172.040,none,0.406376,none,0.108173,none,0.047576,none,-0.021004,none,none
502,ZBRA,513.630,none,1.071062,none,0.371587,none,0.101631,none,0.018201,none,none
503,ZION,57.410,none,0.683278,none,0.266763,none,0.029791,none,-0.059967,none,none


Calculating Momentum Percentiles

In [6]:
intervals = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
            ]

for row in momentum_dataframe.index:
    for time_period in intervals:
    
        price_returns = f'{time_period} Price Return'       
        if momentum_dataframe.loc[row, price_returns] == None:
            momentum_dataframe.loc[row, price_returns] = 0.0  
            
for row in momentum_dataframe.index:
    for time_period in intervals:
    
        price_returns = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        momentum_dataframe.loc[row, percentile_col] = stats.percentileofscore(momentum_dataframe[price_returns], momentum_dataframe.loc[row, price_returns]) / 100
            
            
momentum_dataframe        
        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,152.250,none,0.725821,0.738614,0.257105,0.669307,0.219287,0.893069,0.103362,0.940594,none
1,AAL,22.343,none,0.737275,0.744554,0.437642,0.910891,0.024899,0.261386,-0.029083,0.332673,none
2,AAP,207.122,none,0.430485,0.457426,0.260278,0.673267,0.116394,0.625743,0.059896,0.843564,none
3,AAPL,137.260,none,0.501891,0.526733,0.014254,0.128713,0.114543,0.619802,0.052123,0.811881,none
4,ABBV,113.230,none,0.275424,0.253465,0.144046,0.427723,0.132897,0.691089,-0.010288,0.471287,none
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.800,none,0.386107,0.394059,0.100533,0.312871,0.098295,0.558416,-0.025591,0.358416,none
501,ZBH,172.040,none,0.406376,0.425743,0.108173,0.330693,0.047576,0.348515,-0.021004,0.39802,none
502,ZBRA,513.630,none,1.071062,0.89901,0.371587,0.841584,0.101631,0.570297,0.018201,0.673267,none
503,ZION,57.410,none,0.683278,0.69901,0.266763,0.689109,0.029791,0.285149,-0.059967,0.126733,none


Calculating Momentum Score as Mean of Return Percentiles

In [7]:
from statistics import mean

for row in momentum_dataframe.index:
    momentum_percentiles = []
    for time_period in intervals:
        momentum_percentiles.append(momentum_dataframe.loc[row, f'{time_period} Return Percentile'])
    momentum_dataframe.loc[row, 'Momentum Score'] = mean(momentum_percentiles)

momentum_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,152.250,none,0.725821,0.738614,0.257105,0.669307,0.219287,0.893069,0.103362,0.940594,0.810396
1,AAL,22.343,none,0.737275,0.744554,0.437642,0.910891,0.024899,0.261386,-0.029083,0.332673,0.562376
2,AAP,207.122,none,0.430485,0.457426,0.260278,0.673267,0.116394,0.625743,0.059896,0.843564,0.65
3,AAPL,137.260,none,0.501891,0.526733,0.014254,0.128713,0.114543,0.619802,0.052123,0.811881,0.521782
4,ABBV,113.230,none,0.275424,0.253465,0.144046,0.427723,0.132897,0.691089,-0.010288,0.471287,0.460891
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.800,none,0.386107,0.394059,0.100533,0.312871,0.098295,0.558416,-0.025591,0.358416,0.405941
501,ZBH,172.040,none,0.406376,0.425743,0.108173,0.330693,0.047576,0.348515,-0.021004,0.39802,0.375743
502,ZBRA,513.630,none,1.071062,0.89901,0.371587,0.841584,0.101631,0.570297,0.018201,0.673267,0.74604
503,ZION,57.410,none,0.683278,0.69901,0.266763,0.689109,0.029791,0.285149,-0.059967,0.126733,0.45


Sort stock tickers by Momentum Score and only keep the top 50 tickers

In [8]:
sorted_dataframe = momentum_dataframe.sort_values(by = 'Momentum Score', ascending = False)

In [9]:
sorted_dataframe = sorted_dataframe[:51]
sorted_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
148,DVN,30.710,none,1.863536,0.992079,0.99236,0.99802,0.347354,0.988119,0.154777,0.974257,0.988119
317,MRO,14.160,none,1.475304,0.970297,1.096825,1.0,0.339557,0.986139,0.158736,0.976238,0.983168
175,FANG,98.300,none,1.342828,0.954455,0.981302,0.99604,0.285663,0.952475,0.229775,0.994059,0.974257
174,F,15.589,none,1.603673,0.980198,0.745471,0.986139,0.263515,0.930693,0.170167,0.984158,0.970297
345,NVDA,793.600,none,1.10734,0.918812,0.481376,0.938614,0.525147,1.0,0.236748,0.99604,0.963366
275,LB,73.623,none,4.234641,1.0,0.857653,0.992079,0.265403,0.934653,0.093129,0.920792,0.961881
195,FTNT,248.030,none,0.801802,0.79802,0.621369,0.972277,0.417013,0.994059,0.127054,0.966337,0.932673
149,DXC,39.216,none,1.567136,0.978218,0.693825,0.984158,0.465716,0.99802,0.024681,0.712871,0.918317
173,EXR,169.920,none,0.903377,0.845545,0.457756,0.924752,0.261763,0.926733,0.126689,0.964356,0.915347
426,SYF,49.978,none,1.29463,0.948515,0.473565,0.936634,0.251772,0.920792,0.058574,0.837624,0.910891


Calculate number of shares to buy, $50,000 portfolio size will be used as reference.
Assuming equal distribution of entire portfolio across the 50 stocks.

In [11]:
position_size = float(50000) / len(sorted_dataframe.index) 

for index, row in sorted_dataframe.iterrows():
    sorted_dataframe.loc[index, 'Number of Shares to Buy'] = math.floor(position_size / sorted_dataframe['Price'][index])
sorted_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
148,DVN,30.710,31,1.863536,0.992079,0.99236,0.99802,0.347354,0.988119,0.154777,0.974257,0.988119
317,MRO,14.160,69,1.475304,0.970297,1.096825,1.0,0.339557,0.986139,0.158736,0.976238,0.983168
175,FANG,98.300,9,1.342828,0.954455,0.981302,0.99604,0.285663,0.952475,0.229775,0.994059,0.974257
174,F,15.589,62,1.603673,0.980198,0.745471,0.986139,0.263515,0.930693,0.170167,0.984158,0.970297
345,NVDA,793.600,1,1.10734,0.918812,0.481376,0.938614,0.525147,1.0,0.236748,0.99604,0.963366
275,LB,73.623,13,4.234641,1.0,0.857653,0.992079,0.265403,0.934653,0.093129,0.920792,0.961881
195,FTNT,248.030,3,0.801802,0.79802,0.621369,0.972277,0.417013,0.994059,0.127054,0.966337,0.932673
149,DXC,39.216,24,1.567136,0.978218,0.693825,0.984158,0.465716,0.99802,0.024681,0.712871,0.918317
173,EXR,169.920,5,0.903377,0.845545,0.457756,0.924752,0.261763,0.926733,0.126689,0.964356,0.915347
426,SYF,49.978,19,1.29463,0.948515,0.473565,0.936634,0.251772,0.920792,0.058574,0.837624,0.910891
